In [2]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from lime import lime_image
from skimage.segmentation import slic
from scipy.stats import pearsonr

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cnn_model = load_model("/Users/samarpanpoudel/Desktop/model 1/saved_models/model1.h5")
resnet_model = load_model("/Users/samarpanpoudel/Desktop/Transfer Learning/saved_models/resnet1.h5")
efficientnetbo_model=load_model("/Users/samarpanpoudel/Desktop/EfficientNet-BO/saved_models/efficientnet_bo.h5")

In [4]:
def cnn_predict(images):
    return cnn_model.predict(images)

def resnet_predict(images):
    return resnet_model.predict(images)

def efficientnet_predict(images):
    return efficientnetbo_model.predict(images)

In [5]:
explainer = lime_image.LimeImageExplainer()

def generate_lime_explanation(model_predict, img_array, segmentation):
    explanation = explainer.explain_instance(
        img_array,
        model_predict,
        top_labels=1,
        hide_color=0,
        num_samples=1000,
        segmentation_fn=lambda x: segmentation
    )
    return explanation

In [6]:
def get_importance_vector(explanation, label, num_superpixels):
    importance_dict = dict(explanation.local_exp[label])
    
    vector = np.zeros(num_superpixels)
    
    for superpixel_idx, weight in importance_dict.items():
        vector[superpixel_idx] = weight
        
    return vector

In [7]:
folder_path = "/Users/samarpanpoudel/Desktop/test"

image_files = sorted(os.listdir(folder_path))

correlations1 = []
correlations2=[]
correlations3=[]

for img_name in image_files:
    
    img_path = os.path.join(folder_path, img_name)
    
    img = image.load_img(img_path, target_size=(256,256))
    img_array = image.img_to_array(img)
    
    # Generate segmentation ONCE
    segmentation = slic(img_array, n_segments=50, compactness=10, sigma=1)
    num_superpixels = len(np.unique(segmentation))
    
    # LIME for CNN
    cnn_exp = generate_lime_explanation(cnn_predict, img_array, segmentation)
    cnn_label = cnn_exp.top_labels[0]
    cnn_vector = get_importance_vector(cnn_exp, cnn_label, num_superpixels)
    
    # LIME for ResNet50
    resnet_exp = generate_lime_explanation(resnet_predict, img_array, segmentation)
    resnet_label = resnet_exp.top_labels[0]
    resnet_vector = get_importance_vector(resnet_exp, resnet_label, num_superpixels)

    # LIME for EfficientNet_BO
    efficientnet_exp=generate_lime_explanation(efficientnet_predict, img_array, segmentation)
    efficientnet_label=efficientnet_exp.top_labels[0]
    efficientnet_vector= get_importance_vector(efficientnet_exp, efficientnet_label, num_superpixels)
    
    # Compute correlations
    corr1  = pearsonr(cnn_vector, resnet_vector)
    correlations1.append(corr1)
    
    corr2=pearsonr(cnn_vector, efficientnet_vector)
    correlations2.append(corr2)

    corr3=pearsonr(resnet_vector, efficientnet_vector)
    correlations3.append(corr3)

  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step


  1%|          | 10/1000 [00:00<00:15, 63.12it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


  3%|▎         | 30/1000 [00:00<00:09, 101.50it/s]

...

 98%|█████████▊| 980/1000 [00:17<00:00, 55.96it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step


 99%|█████████▉| 990/1000 [00:17<00:00, 56.54it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


100%|██████████| 1000/1000 [00:17<00:00, 56.35it/s]
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packa

In [8]:
correlations1 = np.array(correlations1)

print("\n LIME Correlation cnn and restnet50:", np.mean(correlations1))
print("Std cnn and restnet50:", np.std(correlations1))

correlations2 = np.array(correlations2)

print("\n LIME Correlation cnn and efficientnet_50:", np.mean(correlations2))
print("Std cnn and efficientnet_50:", np.std(correlations2))

correlations3 = np.array(correlations3)

print("\n LIME Correlation efficientnet_50 and restnet50:", np.mean(correlations3))
print("Std efficientnet_50 and restnet50:", np.std(correlations3))


 LIME Correlation cnn and restnet50: 0.2429152616452376
Std cnn and restnet50: 0.23707913262150382

 LIME Correlation cnn and efficientnet_50: 0.24041612151389322
Std cnn and efficientnet_50: 0.3259988219459394

 LIME Correlation efficientnet_50 and restnet50: 0.2288290077700498
Std efficientnet_50 and restnet50: 0.257079598357974


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
cosine_similarity([cnn_vector],[resnet_vector])

array([[0.30673976]])

In [13]:
cosine_similarity([cnn_vector],[efficientnet_vector])

array([[0.010329]])

In [14]:
cosine_similarity([resnet_vector],[efficientnet_vector])

array([[0.46113134]])